Dieses Notebook dient Identifikation und Extraktion von Nutzeranforderungen.
Mittels OpenAIs ChatGPT4 sollen Anforderungen aus Interviews abgeleitet werden. Diese werden gegen eine von fachlich versierten Personen erstelle Anforderungsliste verglichen, um die Güte der Anforderungsextraktion zu bewerten.
Die Anforderungen werden hinsichtlich ihrer Klassifikation in funktionale und nicht-funktionale Anforderungen, sowie den Inhalt der Anforderung verglichen. Dabei werden nur direkt aus dem Text ableitbare Anforderungen verglichen, implizite Anforderungen werden hier nicht betrachtet.

In [1]:
import os
import openai
import pandas as pd
import datetime
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

# OpenAI API key for SIENA project

openai.api_key  = "YOUR_API_KEY"

In [2]:
# helper function to get the completion from the API

def get_completion_from_messages(messages, model, temperature):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

In [3]:
def text_to_dataframe(text):
    # Split des Textes in einzelne Anforderungen
    requirements = text.split('\n')

    # Initialisierung der Spalten des Dataframes
    columns = ['Klassifikation', 'Anforderung', 'Beschreibung', 'Zitat']

    # Erstellen des Dataframes
    data = pd.DataFrame(columns=columns)

    # Iteration über jede Anforderung
    for requirement in requirements:
        # Split der Anforderung in Klassifikation, Anforderung, Beschreibung und Zitat
        parts = requirement.split('; ')

        # Überprüfen, ob die Anforderung alle erforderlichen Teile enthält
        if len(parts) == 4:
            # Hinzufügen der Anforderung zum Dataframe
            row = pd.Series({
                'Klassifikation': parts[0],
                'Anforderung': parts[1],
                'Beschreibung': parts[2],
                'Zitat': parts[3][1:]
            })
            data = pd.concat([data, row], axis=1)
        else:
            # Ausgabe einer Fehlermeldung für die fehlerhafte Anforderung
            #print("Ungültige Anforderung: " + requirement)
            global req_failures 
            req_failures += 1

    # Dataframe zurückgeben
    return data.T


In [4]:
# interviewauschnitte laden und als dataframe speichern

df_01 = pd.read_excel("SIENA_data_interviews.xlsx")

for question in df_01["Interviewausschnitt"]:
    print(question)
    break

# dataframe für die Anforderungen erstellen

df_02 = pd.DataFrame(columns=['Klassifikation', 'Anforderung', 'Beschreibung', 'Zitat'])

I: Dankeschön Interviewer 2, ich finde das auch total spannend. Du scheinst schon ziemlich konkrete Vorstellungen zu haben, was du brauchen würdest. Klang jetzt zumindest so raus. Und genau, es ist auch unsere Frage, also, wie müssen Informationen optimal aufbereitet werden? Wie müssen sie optimal bereitgestellt werden, damit du sie auch tatsächlich praxistauglich, würde ich jetzt mal verklausulieren, auch konsumieren kannst, um dich entsprechend zu informieren und da auch ein Vertrauen in diese Informationen, die da bereitgestellt werden, entwickeln kannst? Und ohne das jetzt zu sehr irgendwie in eine Form pressen zu wollen, wir sagen nicht ohne Grund, digitales Tool, was entwickelt werden soll. Würde ich mir jetzt von dir gerne mal … #00:18:45#, was du dir vorstellst als Lösung, wie dir genau diese Probleme, die du gerade schon beschrieben hast und diese Qualität, die du beschrieben hast, dann auch liefern kann?
B: Also, ich könnte mir zum Beispiel vorstellen, dass das eine App ist, 

In [5]:
# Iteratives Ausführen der Abfrage für alle Element im df_01["Interviewausschnitt"]

#model = "gpt-4"
model = "gpt-4-0314"
#model = "gpt-3.5-turbo"
#model = "gpt-3.5-turbo-0301"

api_failures = 0
save_failures = 0
req_failures = 0
int_failures = 0
n = 0

for element in df_01["Interviewausschnitt"]:
    
    n_elements = len(df_01["Interviewausschnitt"])
    n += 1

    try:
        text = element

        messages =  [  
        {'role':'system', 'content':'Du bist ein System, das Anforderungen in natürlich sprachlichen Interviews identifiziert und extrahiert.\
        Dir wird vom """user""" ein Interviewausschnitt präsentiert. \
        In dem Interview geht es um die Entwicklung einer Anwendung zur Bereitstellung von medizinischem Fachwissen für Physiotherapeut_Innen. \
        Der Ausschnitt startet mit einer Frage durch einen Interviewer "I".\
        Der Interviewer stellt eine oder mehrere Fragen an den Interviewpartner "B".\
        Der Interviewpartner "B" antwortet auf die Frage.\
        Du analysierst den Interviewausschnitt und versuchst direkte Hinweise auf vom Interviewpartner "B" \
        gewünschte Funktionen der Anwendung zu identifizieren. \
        Ignoriere Anforderungen, die nur aus der Aussage des Interviewers "I" resultieren. \
        Du gibst anschließend eine Liste von Anforderungen aus, die du aus dem Interviewausschnitt extrahiert hast. \
        Die Anforderungen sollen in funktionale und nicht-funktionale Anforderungen unterteilt werden. \
        Du kannst auch eine Anforderung als unklar markieren, wenn du dir nicht sicher bist, ob sie eine Anforderung ist. \
        Es sollen keine impliziten Anforderungen ausgegeben werden. \
        Jede Anforderung soll mit einem Zitat aus dem Interviewausschnitt belegt werden. \
        Die Anforderungen soll nach folgendem Schema formuliert werden: Das System muss/kann/soll + Anforderung. \
        Gib das Ergenis deiner Analyse als CSV aus, nutze als Trennzeichen ";" und folgende Struktur:\
        Funktional/nicht funktional/unklar; Anforderung; Beschreibung der Anforderung; Zitat aus Interviewausschnitt\
        '},
        {'role':'user', 'content':text}]

        try: 
            response = get_completion_from_messages(messages, model = model, temperature=0)
            #print(response[0:100])
        except:
            #print("Fehler beim API Aufruf")
            api_failures += 1

        try:
            # Speichern der Antwort als Textdatei
            now = datetime.datetime.now()
            filename = "/Users/dorianzwanzig/SIENA/responses/response-" + str(model) + "-" + now.strftime("%Y-%m-%d_%H-%M-%S") + ".txt"
            with open(filename, "w") as f:
                f.write(response)
        except:
            #print("Fehler Speichern der Antwort") 
            save_failures += 1  

        try:
            df_02 = df_02.append(text_to_dataframe(response), ignore_index=True)
            df_02.dropna(inplace=True)
            df_02.drop_duplicates(inplace=True)
        except:
            #print("Fehler beim Erweitern des Dataframes")
            req_failures += 1

        print("Fortschritt: " + str(n) + "/" + str(n_elements) + " API-Failures: " + str(api_failures) + 
              " Save-Failures: " + str(save_failures) + " Requirement-Failures: " + str(req_failures) + 
              " Interview-Failures: " + str(int_failures))

    except:
        #print("Fehler beim Verarbeiten des Interviewausschnitts" + str(element[0:200]))
        int_failures += 1

filename_csv = "requirements-" + str(model) + now.strftime("%Y-%m-%d_%H-%M-%S") + ".csv"
df_02.to_csv(filename_csv, index=False)

Fortschritt: 1/15 API-Failures: 0 Save-Failures: 0 Requirement-Failures: 0 Interview-Failures: 0
Fortschritt: 2/15 API-Failures: 0 Save-Failures: 0 Requirement-Failures: 0 Interview-Failures: 0
Fortschritt: 3/15 API-Failures: 0 Save-Failures: 0 Requirement-Failures: 2 Interview-Failures: 0
Fortschritt: 4/15 API-Failures: 0 Save-Failures: 0 Requirement-Failures: 2 Interview-Failures: 0
Fortschritt: 5/15 API-Failures: 0 Save-Failures: 0 Requirement-Failures: 4 Interview-Failures: 0
Fortschritt: 6/15 API-Failures: 0 Save-Failures: 0 Requirement-Failures: 4 Interview-Failures: 0
Fortschritt: 7/15 API-Failures: 0 Save-Failures: 0 Requirement-Failures: 4 Interview-Failures: 0
Fortschritt: 8/15 API-Failures: 0 Save-Failures: 0 Requirement-Failures: 4 Interview-Failures: 0
Fortschritt: 9/15 API-Failures: 0 Save-Failures: 0 Requirement-Failures: 4 Interview-Failures: 0
Fortschritt: 10/15 API-Failures: 0 Save-Failures: 0 Requirement-Failures: 4 Interview-Failures: 0
Fortschritt: 11/15 API-Failur